In [54]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import binom, chi2, chisquare

---
# Ejercicio 5
Calcular una aproximación del *p−valor* de la prueba de que los siguientes datos corresponden
a una distribución binomial con parámetros *(n = 8, p)*, donde p no se conoce:
$$
6, \quad 7,\quad 3, \quad 4, \quad 7, \quad 3, \quad 7, \quad 2, \quad 6, \quad 3, \quad 7, \quad 8, \quad 2, \quad 1, \quad 3, \quad 5, \quad 8, \quad 7
$$

Como en este caso no se conóce el parámetro $p$, entonces primero estimarémos el parámetro $p$, con $\hat{p}$ y luego calcularemos el estadístico en base a esa estimación tal que:
$$
D = sup_{x\in R} |F_e(x) - F_{\hat{p}}(x)|
$$

Sea X una v.a con distribución Binomial con parámetros $n=8$ y $p$ desconocido. i.e $X\sim B(n=8  p)$

Sabemos que para esta v.a su esperanza viene dada por:
$$
E[X] = n\times p
$$

Luego:
$$
\frac{E[X]}{n} = p
$$

Pero como queremos estimar p, i.e $\hat{p}$,  utilizamos un estimador para $E[X]$ tal que:
$$
\begin{align*}
    \hat{p} &= \frac{\bar{X}(18)}{8}\\[0.3cm]
    \hat{p} &= \frac{1}{8} \times \bigg(\frac{1}{18} \times \sum_{i=1}^{18} x_i \bigg)\\[0.3cm]
    \hat{p} &= \frac{1}{8} \times \bigg(\frac{1}{18} \times (6 + 7 + 3  + 4  + 7  + 3  + 7  + 2  + 6  + 3  + 7  + 8  + 2  + 1  + 3  + 5  + 8  + 7)\bigg) \\[0.3cm]
    \hat{p} &= \frac{1}{144} \times 89 \\[0.3cm]
    \hat{p} &\approx 0.6181
\end{align*}
$$

Como ya tenemos una estimación de $p$, ahora lo utilizamos para calcular el estadístico. Por el poder de la paja lo voy
a hacer directamente en el código. 

In [ ]:
def p_estimation(samples:list[float], n:int) -> float:
    """
    Estimación del parámetro p

    Args:
        samples (list[float]): Lista de muestras

    Returns:
        float: p (sombrero)
    """
    hat_p = 0
    bar_x = sum(samples) / len(samples)
    hat_p = (1 / n) * bar_x
    return hat_p

In [46]:
SAMPLES = [
    6, 7, 3,
    4, 7, 3,
    7, 2, 6,
    3, 7, 8,
    2, 1, 3,
    5, 8, 7
]
NSAMPLES = len(SAMPLES)
N = 8

#A falta del parámetro P de los datos, lo estimamos
# a través de las muestras
P_ESTIMATION = round(p_estimation(samples=SAMPLES, n=N), 4)
P_ESTIMATION

0.6181

Ahora como queremos saber si los datos provienen de una variable aleatoria discreta, utilizamos el *Test de Pearson*

In [ ]:
# Calculamos los N_i: Frecuencias Observadas
OBSERVED_FRECUENCIES = np.bincount(SAMPLES, minlength=9)
for i, count in enumerate(OBSERVED_FRECUENCIES):
    print(f"N_{i}: {count}")

N_0: 0
N_1: 1
N_2: 2
N_3: 4
N_4: 1
N_5: 1
N_6: 2
N_7: 5
N_8: 2


[0, 1, 2, 4, 1, 1, 2, 5, 2]

In [ ]:
# Con el parámetro p̂ debo calcular los p_i
PROBABILITY_VALUES = [binom.pmf(k=i, n=N ,p=P_ESTIMATION) for i in range(len(OBSERVED_FRECUENCIES))]
PROBABILITY_VALUES

[np.float64(0.00045247779040564345),
 np.float64(0.005858633616124186),
 np.float64(0.0331874182598645),
 np.float64(0.1074267778288674),
 np.float64(0.21733598905480167),
 np.float64(0.2814042939717681),
 np.float64(0.22772452749927968),
 np.float64(0.10530545052729189),
 np.float64(0.021304431451596998)]

Ahora solo resta calcular el estadístico y realizar el *test de Pearson*

In [52]:
def Pearson_statistic(Nsamples:int, samples:list[int], probability_values:list[float]) -> float:
    statistic = 0
    for sample, prob_i in zip(samples, probability_values):
        statistic += (sample - Nsamples * prob_i) ** 2 / (Nsamples * prob_i)
    return statistic

def Pearson_test(Nsamples: int, samples:list[int], probability_values:list[float], alpha:float):
    T = Pearson_statistic(Nsamples=Nsamples, samples=samples, probability_values=probability_values)
    print(f"🧐 T estadístico:{round(T, 4)}")

    df = len(OBSERVED_FRECUENCIES) - 1 - 1

    p_value = round(1 - chi2.cdf(round(T, 4), df=df), 4)

    print(f"☝️ p-valor obtenido: {p_value}")

    if p_value > alpha:
        print(f"☝️😲 Cómo {p_value} > {alpha}")
        print("\t 😒 No hay evidencia suficiente para rechazar Ho")
    else:
        print(f"☝️😲 Cómo {p_value} <= {alpha}")
        print(f"\t 🔴 Se rechaza Ho con una confianza del {100 * (1-alpha)}%")

In [70]:
ALPHA = 1 - 0.95
Pearson_test(
        Nsamples=NSAMPLES,
        samples=OBSERVED_FRECUENCIES.tolist(),
        probability_values=PROBABILITY_VALUES,
        alpha=ALPHA
)

🧐 T estadístico:31.5019
☝️ p-valor obtenido: 0.0001
☝️😲 Cómo 0.0001 <= 0.050000000000000044
	 🔴 Se rechaza Ho con una confianza del 95.0%


# 😲 Estimación del p-valor

In [75]:
def p_value_estimation(Nsim:int, Nsamples:int, samples:list[int], probability_values:list[float],  t0:float):
    p_value = 0
    possible_frequencies = list(range(0, len(samples)))
    for _ in range(Nsim):
        #Generamos Nsamples muestras de las posibles frecuencias observadas,
        #cada una con probabilidad p_i
        X_samples = np.random.choice(a=possible_frequencies,  p=probability_values, size=Nsamples)

        N_i = np.bincount(X_samples, minlength=len(samples))
        
        T = Pearson_statistic(Nsamples=Nsamples, samples=N_i, probability_values=probability_values)

        if T > t0:
            p_value += 1

    return p_value / Nsim

In [82]:
NSIM = 10_000
T = 31.5019
p_value = p_value_estimation(
            Nsim=NSIM,
            Nsamples=NSAMPLES,
            samples=OBSERVED_FRECUENCIES,
            probability_values=PROBABILITY_VALUES,
            t0=T)

print(f"☝️ p-valor obtenido: {round(p_value, 4)}")

☝️ p-valor obtenido: 0.0146


# Implementación chiteada de SCIPY 🐍

In [62]:
# Calcular frecuencias esperadas
EXPECTED_FREQUENCIES = [NSAMPLES * p_i for p_i in PROBABILITY_VALUES]

# Ejecutar el test de chi-cuadrado con 1 grado de libertad menos (por estimar p̂)
test_result = chisquare(f_obs=OBSERVED_FRECUENCIES, f_exp=EXPECTED_FREQUENCIES, ddof=1)
T = round(test_result.statistic, 4)
p_value = round(test_result.pvalue, 4)

print(f"🧪 T estadístico (scipy): {T:.4f}")
print(f"📉 p-valor (scipy): {p_value:.4f}")


if p_value > ALPHA:
    print(f"☝️😲 Cómo {p_value} > {ALPHA}")
    print("\t 😒 No hay evidencia suficiente para rechazar Ho")
else:
    print(f"☝️😲 Cómo {p_value} <= {ALPHA}")
    print(f"\t 🔴 Se rechaza Ho con una confianza del {100 * (1-ALPHA)}%")

🧪 T estadístico (scipy): 31.5019
📉 p-valor (scipy): 0.0001
☝️😲 Cómo 0.0001 <= 0.050000000000000044
	 🔴 Se rechaza Ho con una confianza del 95.0%
